# [讨论纪要] 关于报警的处理 (20180421)

### 背景

                                [  V  o  e  r  k  e  C  l  o  u  d  ]
                                [    ┌→ ...原始报警 → 业务报警    ]
                                     |  ┌————————————————┘ 
                                     |  |
                                     |  ↓
                               [MQTT BROKER]
                               ↑          \
                              /             路1—————、
                             /                      ↓
                  [   病   房   分   机   ] —— 路2→ [主机]
                    ↑   |    ↑    |   ↑
                    |    |    |    |    |
                    ...[  输  液  报  警 ]

### 问题

	注1: 以下分机和主机所在分组皆假设为0
    注2: /voerka/groups/0/主机节点路径/# 包含了 /voerka/groups/0/分/机/节/点/路/径/

	**原来的处理报警方式:**
		分机向主题 /voerka/devices/分机sn/alarm 发布报警信息
		
		服务器因为订阅了 /voerka/devices/# 而收到该信息
		服务器的服务处理该(相对要转化成的业务而言的)原始报警信息, 将其转化为业务报警信息
		服务向主题 /voerka/groups/0/分/机/节/点/路/径/ 发布该业务报警信息
        
		主机因为订阅了 /voerka/groups/0/主机节点/# 而收到该业务报警信息并渲染（注2）
        
    问题在于: 主机要依赖于服务器才能收到业务报警信息
    	导致两个结果:
    		1. 脱机状态下不能获得任何报警
    		2. 在主机不需要业务报警(只需要原始报警)的情况下(服务器只监控), 本应该直接发送给主机, 还要绕远路(不经济)
            
### 方法  

    **更改原来的处理报警方式为:**
		
        分机向主题 /voerka/groups/0/分/机/节/点/路/径/$alarm 发布报警信息
        
		主机因为订阅了 /voerka/groups/0/主机节点路径/# 而收到该报警信息（注2）
        
		服务器因为订阅了 /voerka/groups/#/$alarm 而收到该信息
		服务器的服务处理该原始报警信息, 将其转化为业务报警信息 或 仅做监控
		
		如果业务层逻辑认为需要处理成业务报警并发布, 则服务向主题 /voerka/groups/0/分/机/节/点/路/径 发布该业务报警信息
		同时 sync 字段设为 True
        
			主机因为订阅了 /voerka/groups/0/主机节点路径/# 而 又一次 收到该报警信息
			分机因为订阅了 /voerka/groups/0/分/机/节/点/路/径/# 而收到该报警信息
				判断 sync 字段为 True 则不做处理
        
        如果仅做监控, 则不转化也不发布
		
			主机没有再次收到一条业务报警信息
        
    **效果:**
        1. 脱机状态下(无服务器), 主机仍然能收到分机的报警(可能只是原始报警, 这取决于主机和分机协调)
        2. 从而服务器如果只监控的话, 主机可以直接收到分机报警(可能只是原始报警, 这取决于主机和分机协调)
        
    **问题:**
    	1. 主机是有可能收到两条报警信息, 是否是冗余?
    		主机收到的并非重复的报警信息, 一条是原始报警, 一条是业务报警
        
    	2. 主机将有可能收到两条报警信息, 如何处理?
    		对于原始报警信息, 主机按照原始报警信息的处理方式处理, 对于业务报警信息, 则按照相应的处理方式处理
    		或其他可能的处理方式, 以协调两种报警信息
        
        3. 分机收到 sync 为 True 的信息则抛弃, 是否不经济?
        	暂设计如此
        
        4. 
